Code to (re)produce results in the paper 
"Manipulating the Online Marketplace of Ideas" 
by Xiaodan Lou, Alessandro Flammini, and Filippo Menczer
https://arxiv.org/abs/1907.06130

Notes:
* Need Python 3.6 or later; eg: `module load python/3.6.6`
* Remember link direction is following, opposite of info spread!
* For large `n_humans`, it's much faster to run the simulations in parallel on a server or cluster, eg, one process for each combination of parameters (gamma, phi, mu...)

Parameters and default values:
```
n_humans = 1000 # 10k for paper
beta = 0.1 # bots/humans ratio; 0.1 for paper
p = 0.5 # for network clustering; 0.5 for paper
k_out = 3 # average no. friends within humans & bots; 3 for paper
alpha = 15 # depth of feed; 15 for paper
mu = 0.5 # average prob of new meme vs retweet; 0.75 for earlier draft
phi = 1 # bot deception >= 1: meme fitness higher than quality
gamma = 0.1 # infiltration: probability that a human follows each bot
epsilon = 0.01 # threshold used to check for steady-state convergence
n_runs = 10 # number of simulations to average results
csvfile = 'results.csv' # to save results for plotting
```

In [ ]:
import os
import networkx as nx
import random
import numpy
import numpy as np
import math
import statistics
import csv
import matplotlib.pyplot as plt
from scipy import stats
from operator import itemgetter
from collections import defaultdict
import sys
import fcntl
import time
import pickle
import bot_model

%matplotlib inline
assert(nx.__version__ >= '2.4')

# Loop over parameters and simulation runs

In [3]:
def main_exp(preferential_targeting_flag, phis, gammas):
    
    condition = "prefer" if preferential_targeting_flag else "random"
    save_dir = "results/" + condition
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    for phi in phis:
      for gamma in gammas:
        
        valid_tracked_memes_all = []
        bad_memes_selected_time_all = {}
        avg_quality_all = []
        avg_diversity_all = []
        
        for sim in range(n_runs):
          print('Running Simulation ', sim, ' for phi = ', phi, ', gamma = ', gamma, ' ...', flush=True)

          # simulation start
          q, q_net = bot_model.simulation(preferential_targeting_flag, return_net=True, count_forgotten=True, track_meme=True, phi=phi, gamma=gamma) 

          #### statistic current nth-run data ####

          ## avg quality ##
          avg_quality_all.append(q)
          ## end avg quality ##

          ## tracked memes ##
          for meme in bot_model.track_memes.popularity:
            valid_tracked_memes_all.append((meme[0], bot_model.track_memes.popularity[meme]))
          ## end tracked memes ##

          ## bad memes selected ##
          for meme_id, selected_time in bot_model.track_memes.bad_popularity.items():
            if meme_id not in bad_memes_selected_time_all:
              bad_memes_selected_time_all[meme_id] = [0, 0]
            bad_memes_selected_time_all[meme_id][0] += selected_time[0]
            bad_memes_selected_time_all[meme_id][1] += selected_time[1]
          ## end bad memes selected ##

          ## avg diversity ##
          for agent in q_net.nodes:
            qualities = []
            for m in q_net.nodes[agent]['feed']:
              qualities.append(m[0])
            unique_qua, unique_qua_cnt = np.unique(qualities, return_counts=True)
            portion_of_qua = unique_qua_cnt / np.sum(unique_qua_cnt)
            diversity = - np.sum(portion_of_qua * np.log(portion_of_qua))
            avg_diversity_all.append(diversity)
            # could in theory calculate diversity based on fitness...
          ## end avg diversity ##

          #### end statistic current nth-run data ####

        # average the counts of bad meme selections across runs
        for meme_id, selected_time in bad_memes_selected_time_all.items():
          bad_memes_selected_time_all[meme_id][0] /= n_runs
          bad_memes_selected_time_all[meme_id][1] /= n_runs

        # save avg quality
        fp = open("{}/avg_quality_{}_phi{}_gamma{}.pkl".format(save_dir, condition, phi, gamma), "wb")
        pickle.dump(np.mean(avg_quality_all), fp)
        fp.close()

        # save tracked memes
        fp = open("{}/tracked_memes_{}_phi{}_gamma{}.pkl".format(save_dir, condition, phi, gamma), "wb")
        pickle.dump(valid_tracked_memes_all, fp)
        fp.close()

        # save bad meme selected times
        fp = open("{}/bad_memes_selected_time_{}_phi{}_gamma{}.pkl".format(save_dir, condition, phi, gamma), "wb")
        pickle.dump(bad_memes_selected_time_all, fp)
        fp.close()

        # save avg diversity
        fp = open("{}/avg_diversity_{}_phi{}_gamma{}.pkl".format(save_dir, condition, phi, gamma), "wb")
        pickle.dump(np.mean(avg_diversity_all), fp)
        fp.close()

        # save kendall
        quality, number_selected = zip(*valid_tracked_memes_all)
        kendall_tau, _ = stats.kendalltau(quality, number_selected)
        fp = open("{}/kendall_{}_phi{}_gamma{}.pkl".format(save_dir, condition, phi, gamma), "wb")
        pickle.dump(kendall_tau, fp)
        fp.close()  

# Main experiment

In [1]:
phis = [1, 5, 10] 
gammas = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0] 
main_exp(False, phis, gammas)
#main_exp(True, phis, gammas)

# Plots for figures

In [ ]:
condition = "random"
save_dir = "results/" + condition

In [ ]:
# Main plots of average quality, diversity, and Kendall tau as functions of gamma and phi params

xs = phis
ys = gammas
phis1 = phis
phis2 = phis
wires = gammas
new_wires = gammas
cmap = None
xlabel = '$\\phi$'
ylabel = '$\\gamma$'

kendall_pic_title = 'Discriminative power'
avg_quality_pic_title = 'Average Quality'
diversity_pic_title = 'Diversity'

figure = plt.figure(figsize=(13, 15), facecolor='w')
markers = ["o", "s", "^"]

### 1. average quality ###
file_template = "{}/avg_quality_{}_phi{}_gamma{}.pkl"

# distr plot
ax = figure.add_subplot(3,2,1)
for idx, phi in enumerate(phis1):
    avg_qualities = []
    stds = []
    for gamma in wires:
        fname = file_template.format(save_dir, condition, phi, gamma)
        fp = open(fname, "rb")
        data = pickle.load(fp)
        fp.close()
        avg_qualities.append(np.mean(data))
        stds.append(np.std(data))
    ax.plot(new_wires, avg_qualities, marker=markers[idx], label='$\\phi$:'+str(h))

ax.set_xlabel('$\\gamma$', fontsize=14)
ax.set_ylabel('Average quality', fontsize=14)
ax.set_xscale('log')
ax.set_xlim((new_wires[0], new_wires[-1]))
ax.set_xlim((new_wires[0], new_wires[-1]))
ax.legend(loc='upper right', fontsize=14)

# heatmap plot
ax = figure.add_subplot(3,2,2)
grid = np.zeros((len(wires), len(phis2)))
for i, gamma in enumerate(wires):
    for j, phi in enumerate(phis2):
        fname = file_template.format(save_dir, condition, phi, gamma)
        fp = open(fname, "rb")
        data = pickle.load(fp)
        fp.close()
        grid[i, j] = np.mean(data)
bot_model.draw_heatmap(ax, grid, xs, ys, xlabel, ylabel, cmap, avg_quality_pic_title)


### 2. average diversity ###
file_template = "{}/avg_diversity_{}_phi{}_gamma{}.pkl"

# distr plot
ax = figure.add_subplot(3,2,3)
for idx, phi in enumerate(phis1):
    avg_diversities = []
    stds = []
    for gamma in wires:
        fname = file_template.format(save_dir, condition, phi, gamma)
        fp = open(fname, "rb")
        data = pickle.load(fp)
        fp.close()
        avg_diversities.append(np.mean(data))
        stds.append(np.std(data))
    ax.plot(new_wires, avg_diversities, marker=markers[idx], label='$\\phi$:'+str(h))

ax.set_xlabel('$\\gamma$', fontsize=14)
ax.set_ylabel('Diversity', fontsize=14)
ax.set_xscale('log')
ax.set_xlim((new_wires[0], new_wires[-1]))
ax.set_xlim((new_wires[0], new_wires[-1]))

# heatmap plot
ax = figure.add_subplot(3,2,4)
grid = np.zeros((len(wires), len(phis2)))
for i, gamma in enumerate(wires):
    for j, phi in enumerate(phis2):
        fname = file_template.format(save_dir, condition, phi, gamma)
        fp = open(fname, "rb")
        data = pickle.load(fp)
        fp.close()
        grid[i, j] = np.mean(data)
bot_model.draw_heatmap(ax, grid, xs, ys, xlabel, ylabel, cmap, diversity_pic_title)

### 3. kendall ###
file_template = "{}/kendall_{}_phi{}_gamma{}.pkl"

# distr plot
ax = figure.add_subplot(3,2,5)
for idx, phi in enumerate(phis1):
    kendalls = []
    stds = []
    for gamma in wires:
        fname = file_template.format(save_dir, condition, phi, gamma)
        fp = open(fname, "rb")
        data = pickle.load(fp)
        fp.close()
        kendalls.append(np.mean(data))
        stds.append(np.std(data))
    ax.plot(new_wires, kendalls, marker=markers[idx], label='$\\phi$:'+str(h))

ax.set_xlabel('$\\gamma$', fontsize=14)
ax.set_ylabel('Discriminative power', fontsize=14)
ax.set_xscale('log')
ax.set_xlim((new_wires[0], new_wires[-1]))
ax.set_xlim((new_wires[0], new_wires[-1]))
# ax.legend(loc='lower left', fontsize=14)

# heatmap plot
ax = figure.add_subplot(3,2,6)
grid = np.zeros((len(wires), len(phis2)))
for i, gamma in enumerate(wires):
    for j, phi in enumerate(phis2):
        fname = file_template.format(save_dir, condition, phi, gamma)
        fp = open(fname, "rb")
        data = pickle.load(fp)
        fp.close()
        grid[i, j] = np.mean(data)
bot_model.draw_heatmap(ax, grid, xs, ys, xlabel, ylabel, cmap, kendall_pic_title)

### 4. save plot ###
plt.subplots_adjust(left=0.1, right=0.95, top=0.95, bottom=0.05, wspace=0.3, hspace=0.3)
plt.savefig(save_dir + "/all_distr_heatmap.png")

In [ ]:
# popularity distribution plots

file_template = "{}/tracked_memes_{}_phi{}_gamma{}.pkl"

fig, axs = plt.subplots(2, 3, figsize=(14, 8))
for i, phi in enumerate([1, 10]):
    for j, gamma in enumerate([0.001, 0.005, 0.01]):
        fname = file_template.format(save_dir, condition, phi, gamma)
        fp = open(fname, "rb")
        data = pickle.load(fp)
        fp.close()

        quality, number_selected = zip(*data)

        low_quality_pop = []
        high_quality_pop = []
        for qua, pop in zip(quality, number_selected):
            if qua > 0:
                high_quality_pop.append(pop)
            else:
                low_quality_pop.append(pop)

        count = bot_model.get_count(high_quality_pop)
        distr, sum_ = bot_model.get_distr(count)
        h_mids, h_heights = bot_model.getbins(distr, sum_)

        count = bot_model.get_count(low_quality_pop)
        distr, sum_ = bot_model.get_distr(count)
        l_mids, l_heights = bot_model.getbins(distr, sum_)

        h_dict = defaultdict(list)
        for hm, hh in zip(h_mids, h_heights):
            h_dict[hm].append(hh)
        l_dict = defaultdict(list)
        for lm, lh in zip(l_mids, l_heights):
            l_dict[lm].append(lh)

        hs = []
        for k, v in h_dict.items():
            hs.append([k, np.mean(v)])
        h_mids, h_heights = zip(*sorted(hs, key=lambda x:x[0]))
        ls = []
        for k, v in l_dict.items():
            ls.append([k, np.mean(v)])
        l_mids, l_heights = zip(*sorted(ls, key=lambda x:x[0]))

        ax = axs[i][j]
        ax.loglog(h_mids, h_heights, marker='s', label='high quality')
        ax.loglog(l_mids, l_heights, marker='^', label='low quality')
        ax.set_xlabel('popularity', fontsize=14)
        ax.set_ylabel('P(popularity)', fontsize=14)
        ax.tick_params(labelsize=14)
        ax.annotate('$\\gamma={}$\n$\\phi={}$'.format(gamma, phi), xy=(0.05, 0.05), xycoords='axes fraction', fontsize=12)
        if i == 0 and j == 0:
            ax.legend(loc="upper right", fontsize=15)

plt.subplots_adjust(left=0.08, right=0.92, top=0.92, wspace=0.3, hspace=0.3)
plt.show()
plt.savefig(save_dir + "meme_quality_random_distr.png")
plt.close()


In [ ]:
# amplification plot(s)

file_template = "{}/bad_memes_selected_time_{}_phi{}_gamma{}.pkl"

for i, phi in enumerate([1]):
    plt.figure(figsize=(10, 5))
    for j, gamma in enumerate([0.001]): #[0.5]
        fname = file_template.format(save_dir, condition, phi, gamma)
        fp = open(fname, "rb")
        data = pickle.load(fp)
        fp.close()
        
        good_selected = []
        bad_selected = []
        for _, value in data.items():
            if value[0] <= 0 or value[1] <= 0:
                continue
            good_selected.append(value[0])
            bad_selected.append(value[1])

        count = dict([val for val in zip(bad_selected, good_selected)])
        distr_x, distr_y = bot_model.get_distr(count)
        mids, heights = bot_model.getbins(distr_x, distr_y)
        ratios = [np.log(height_)/np.log(mid_) for height_, mid_ in zip(heights, mids)]

        plt.subplot(121)
        plt.loglog(mids, heights, marker='o', label='$\\gamma$:'+str(gamma))
        plt.subplot(122)
        plt.plot(mids, ratios, marker='o', label='$\\gamma$:'+str(gamma))
        plt.xscale('log')
    
    # save fig
    plt.subplot(121)
    plt.loglog([min(mids), max(mids)], [min(mids), max(mids)], '--')
    plt.xlabel("Bot posts per meme", fontsize=14)
    plt.ylabel("Human posts per meme", fontsize=14)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.margins(0.1)
    plt.legend(loc='best', fontsize=14)

    plt.subplot(122)
    plt.xlabel("Bot posts per meme", fontsize=14)
    plt.ylabel("Exponent $\\eta$", fontsize=14)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.margins(0.1)
    plt.legend(loc='best', fontsize=14)

    plt.subplots_adjust(left=0.1, bottom=0.14, wspace=0.4)
    plt.show()
    plt.savefig(save_dir + "bad_meme_selected_random_distr_{}".format(phi))
    plt.close()